# 📊 M5 Data Exploration for Inventory Optimization

This notebook explores the M5 Walmart dataset to understand demand patterns, sales trends, and data characteristics relevant for inventory optimization.

## Objectives
1. Load and inspect the M5 dataset
2. Analyze demand patterns and distributions
3. Identify seasonality and trends
4. Examine price variations
5. Explore store and category characteristics

In [1]:
# Setup
import sys
from pathlib import Path

# Add project root to path
project_root = Path.cwd().parent.parent
sys.path.insert(0, str(project_root))

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from datetime import datetime

# Import project modules
from src.data import DataLoader, DemandCalculator
from src.utils import load_config, setup_logging

# Configure display
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 50)
plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("Set2")

# Set figure size
plt.rcParams['figure.figsize'] = (14, 6)

print("✅ Setup complete!")

✅ Setup complete!


## 1. Load Configuration and Data

In [11]:
# Load configuration
config = load_config(project_root / 'config/config.yaml')
print("Configuration loaded successfully")

# Initialize data loader with absolute path
# The config path is relative to project-002's parent directory (data-science-portfolio)
# which contains both project-001 and project-002
raw_data_path = (project_root / config['data']['raw_data_path']).resolve()
loader = DataLoader(str(raw_data_path))
print(f"Data path: {raw_data_path}")

Configuration loaded successfully
Data path: /Users/godsonkurishinkal/Projects/data-science-portfolio/project-001-demand-forecasting-system/data/raw


In [12]:
# Load raw data
calendar, sales_train, sell_prices = loader.load_raw_data()

print(f"Calendar shape: {calendar.shape}")
print(f"Sales train shape: {sales_train.shape}")
print(f"Sell prices shape: {sell_prices.shape}")

Calendar shape: (1969, 14)
Sales train shape: (30490, 1947)
Sell prices shape: (6841121, 4)


## 2. Explore Calendar Data

In [13]:
# Inspect calendar
print("Calendar columns:")
print(calendar.columns.tolist())
print("\nFirst few rows:")
calendar.head()

Calendar columns:
['date', 'wm_yr_wk', 'weekday', 'wday', 'month', 'year', 'd', 'event_name_1', 'event_type_1', 'event_name_2', 'event_type_2', 'snap_CA', 'snap_TX', 'snap_WI']

First few rows:


,date,wm_yr_wk,weekday,wday,month,year,d,event_name_1,event_type_1,event_name_2,event_type_2,snap_CA,snap_TX,snap_WI
0,2011-01-29,11101,Saturday,1,1,2011,d_1,NaN,NaN,NaN,NaN,0,0,0
1,2011-01-30,11101,Sunday,2,1,2011,d_2,NaN,NaN,NaN,NaN,0,0,0
2,2011-01-31,11101,Monday,3,1,2011,d_3,NaN,NaN,NaN,NaN,0,0,0
3,2011-02-01,11101,Tuesday,4,2,2011,d_4,NaN,NaN,NaN,NaN,1,1,0
4,2011-02-02,11101,Wednesday,5,2,2011,d_5,NaN,NaN,NaN,NaN,1,0,1


In [14]:
# Calendar date range and events
calendar['date'] = pd.to_datetime(calendar['date'])

print(f"Date range: {calendar['date'].min()} to {calendar['date'].max()}")
print(f"Total days: {len(calendar)}")
print(f"\nEvent types: {calendar['event_type_1'].value_counts()}")
print(f"\nSNAP states: {calendar[['snap_CA', 'snap_TX', 'snap_WI']].sum()}")

Date range: 2011-01-29 00:00:00 to 2016-06-19 00:00:00
Total days: 1969

Event types: event_type_1
Religious    55
National     52
Cultural     37
Sporting     18
Name: count, dtype: int64

SNAP states: snap_CA    650
snap_TX    650
snap_WI    650
dtype: int64


## 3. Explore Sales Data

In [15]:
# Sales train structure
print("Sales train columns (first 10):")
print(sales_train.columns[:10].tolist())
print(f"\nTotal products: {len(sales_train)}")
print(f"Total stores: {sales_train['store_id'].nunique()}")
print(f"Total categories: {sales_train['cat_id'].nunique()}")
print(f"Total departments: {sales_train['dept_id'].nunique()}")

sales_train.head()

Sales train columns (first 10):
['id', 'item_id', 'dept_id', 'cat_id', 'store_id', 'state_id', 'd_1', 'd_2', 'd_3', 'd_4']

Total products: 30490
Total stores: 10
Total categories: 3
Total departments: 7


id        item_id    dept_id   cat_id store_id  \
0  HOBBIES_1_001_CA_1_evaluation  HOBBIES_1_001  HOBBIES_1  HOBBIES     CA_1   
1  HOBBIES_1_002_CA_1_evaluation  HOBBIES_1_002  HOBBIES_1  HOBBIES     CA_1   
2  HOBBIES_1_003_CA_1_evaluation  HOBBIES_1_003  HOBBIES_1  HOBBIES     CA_1   
3  HOBBIES_1_004_CA_1_evaluation  HOBBIES_1_004  HOBBIES_1  HOBBIES     CA_1   
4  HOBBIES_1_005_CA_1_evaluation  HOBBIES_1_005  HOBBIES_1  HOBBIES     CA_1   

  state_id  d_1  d_2  d_3  d_4  d_5  d_6  d_7  d_8  d_9  d_10  d_11  d_12  \
0       CA    0    0    0    0    0    0    0    0    0     0     0     0   
1       CA    0    0    0    0    0    0    0    0    0     0     0     0   
2       CA    0    0    0    0    0    0    0    0    0     0     0     0   
3       CA    0    0    0    0    0    0    0    0    0     0     0     0   
4       CA    0    0    0    0    0    0    0    0    0     0     0     0   

   d_13  d_14  d_15  d_16  d_17  d_18  d_19  d_20  d_21  d_22  d_23  d_24  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_25  d_26  d_27  d_28  d_29  d_30  d_31  d_32  d_33  d_34  d_35  d_36  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     0     0     0     0     0     0     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_37  d_38  d_39  d_40  d_41  d_42  d_43  d_44  d_45  d_46  d_47  d_48  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     2     0     0     0     2     0     1     0     0     0     0     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_49  d_50  d_51  d_52  d_53  d_54  d_55  d_56  d_57  d_58  d_59  d_60  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     2     0     1     0     0     1     1     1     0     2     3   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_61  d_62  d_63  d_64  d_65  d_66  d_67  d_68  d_69  d_70  d_71  d_72  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     1     0     0     0     0     0     1     0     0     1     1     0   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_73  d_74  d_75  d_76  d_77  d_78  d_79  d_80  d_81  d_82  d_83  d_84  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   
3     0     1     0     0     1     2     3     0     2     0     0     2   
4     0     0     0     0     0     0     0     0     0     0     0     0   

   d_85  d_86  d_87  d_88  d_89  d_90  d_91  d_92  d_93  d_94  d_95  d_96  \
0     0     0     0     0     0     0     0     0     0     0     0     0   
1     0     0     0     0     0     0     0     0     0     0     0     0   
2     0     0     0     0     0     0     0     0     0     0     0     0   


In [16]:
# Hierarchy breakdown
print("Store distribution:")
print(sales_train['store_id'].value_counts())
print("\nCategory distribution:")
print(sales_train['cat_id'].value_counts())
print("\nDepartment distribution:")
print(sales_train['dept_id'].value_counts())

Store distribution:
store_id
CA_1    3049
CA_2    3049
CA_3    3049
CA_4    3049
TX_1    3049
TX_2    3049
TX_3    3049
WI_1    3049
WI_2    3049
WI_3    3049
Name: count, dtype: int64

Category distribution:
cat_id
FOODS        14370
HOUSEHOLD    10470
HOBBIES       5650
Name: count, dtype: int64

Department distribution:
dept_id
FOODS_3        8230
HOUSEHOLD_1    5320
HOUSEHOLD_2    5150
HOBBIES_1      4160
FOODS_2        3980
FOODS_1        2160
HOBBIES_2      1490
Name: count, dtype: int64


In [17]:
# Sample sales data (last 90 days)
day_cols = [col for col in sales_train.columns if col.startswith('d_')]
last_90_days = day_cols[-90:]

sample_sales = sales_train[['item_id', 'store_id', 'cat_id'] + last_90_days]
sample_sales['total_sales_90d'] = sample_sales[last_90_days].sum(axis=1)
sample_sales['avg_daily_sales'] = sample_sales[last_90_days].mean(axis=1)
sample_sales['std_daily_sales'] = sample_sales[last_90_days].std(axis=1)

print("Sales statistics (last 90 days):")
sample_sales[['total_sales_90d', 'avg_daily_sales', 'std_daily_sales']].describe()

Sales statistics (last 90 days):


/var/folders/0f/g1brb1zx6vvfxtgphq82592r0000gn/T/ipykernel_37741/107408496.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_sales['total_sales_90d'] = sample_sales[last_90_days].sum(axis=1)
/var/folders/0f/g1brb1zx6vvfxtgphq82592r0000gn/T/ipykernel_37741/107408496.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  sample_sales['avg_daily_sales'] = sample_sales[last_90_days].mean(axis=1)
/var/folders/0f/g1brb1zx6vvfxtgphq82592r0000gn/T/ipykernel_37741/107408496.py:8: SettingWithCopyWarning: 
A val

,total_sales_90d,avg_daily_sales,std_daily_sales
count,30490.000000,30490.000000,30490.000000
mean,125.633716,1.395930,1.354189
std,264.655879,2.940621,1.625500
min,0.000000,0.000000,0.000000
25%,24.000000,0.266667,0.572028
50%,56.000000,0.622222,0.924085
75%,126.000000,1.400000,1.527117
max,10259.000000,113.988889,43.286101


## 4. Process and Analyze Detailed Sales

In [18]:
# Process data for one store (faster for exploration)
full_data = loader.process_data()

# Filter to CA_1 store for detailed analysis
store_data = loader.filter_data(
    full_data,
    stores=['CA_1'],
    start_date='2015-01-01',
    end_date='2016-03-27'
)

print(f"Filtered data shape: {store_data.shape}")
print(f"Date range: {store_data['date'].min()} to {store_data['date'].max()}")

Filtered data shape: (1378148, 23)
Date range: 2015-01-01 00:00:00 to 2016-03-27 00:00:00


In [ ]:
# Daily sales trends
daily_sales = store_data.groupby('date')['sales'].sum().reset_index()

fig, axes = plt.subplots(2, 1, figsize=(16, 10))

# Total daily sales
axes[0].plot(daily_sales['date'], daily_sales['sales'], linewidth=1, alpha=0.7)
axes[0].set_title('Daily Total Sales - CA_1 Store', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Total Sales (units)')
axes[0].grid(True, alpha=0.3)

# 7-day moving average
daily_sales['ma7'] = daily_sales['sales'].rolling(window=7).mean()
daily_sales['ma28'] = daily_sales['sales'].rolling(window=28).mean()

axes[1].plot(daily_sales['date'], daily_sales['sales'], linewidth=1, alpha=0.3, label='Daily')
axes[1].plot(daily_sales['date'], daily_sales['ma7'], linewidth=2, label='7-day MA')
axes[1].plot(daily_sales['date'], daily_sales['ma28'], linewidth=2, label='28-day MA')
axes[1].set_title('Sales with Moving Averages', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('Sales (units)')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Day of week patterns
store_data['day_of_week'] = store_data['date'].dt.dayofweek
store_data['day_name'] = store_data['date'].dt.day_name()

dow_sales = store_data.groupby('day_name')['sales'].agg(['mean', 'sum', 'std']).reset_index()
dow_order = ['Monday', 'Tuesday', 'Wednesday', 'Thursday', 'Friday', 'Saturday', 'Sunday']
dow_sales['day_name'] = pd.Categorical(dow_sales['day_name'], categories=dow_order, ordered=True)
dow_sales = dow_sales.sort_values('day_name')

fig, ax = plt.subplots(figsize=(12, 6))
x = np.arange(len(dow_sales))
ax.bar(x, dow_sales['mean'], alpha=0.7, color='steelblue', edgecolor='black')
ax.errorbar(x, dow_sales['mean'], yerr=dow_sales['std'], fmt='none', color='red', capsize=5)
ax.set_xticks(x)
ax.set_xticklabels(dow_sales['day_name'], rotation=45)
ax.set_title('Average Daily Sales by Day of Week', fontsize=14, fontweight='bold')
ax.set_xlabel('Day of Week')
ax.set_ylabel('Average Sales (units)')
ax.grid(True, alpha=0.3, axis='y')

for i, v in enumerate(dow_sales['mean']):
    ax.text(i, v + 0.5, f'{v:.1f}', ha='center', va='bottom', fontweight='bold')

plt.tight_layout()
plt.show()

## 5. Price Analysis

In [ ]:
# Price distribution
price_stats = store_data.groupby('item_id')['sell_price'].agg(['mean', 'min', 'max', 'std']).reset_index()
price_stats['price_range'] = price_stats['max'] - price_stats['min']
price_stats['price_cv'] = price_stats['std'] / price_stats['mean']

print("Price statistics:")
print(price_stats[['mean', 'min', 'max', 'price_range', 'price_cv']].describe())

In [ ]:
# Price distribution by category
cat_prices = store_data.groupby(['cat_id', 'item_id'])['sell_price'].mean().reset_index()

fig, axes = plt.subplots(1, 3, figsize=(18, 5))

for i, cat in enumerate(['FOODS', 'HOBBIES', 'HOUSEHOLD']):
    cat_data = cat_prices[cat_prices['cat_id'] == cat]
    axes[i].hist(cat_data['sell_price'], bins=50, alpha=0.7, color=f'C{i}', edgecolor='black')
    axes[i].set_title(f'{cat} Price Distribution', fontweight='bold')
    axes[i].set_xlabel('Price ($)')
    axes[i].set_ylabel('Frequency')
    axes[i].axvline(cat_data['sell_price'].mean(), color='red', linestyle='--', 
                    linewidth=2, label=f'Mean: ${cat_data["sell_price"].mean():.2f}')
    axes[i].legend()
    axes[i].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 6. Demand Statistics

In [19]:
# Calculate comprehensive demand statistics
calc = DemandCalculator()
demand_stats = calc.calculate_demand_statistics(
    store_data,
    group_cols=['store_id', 'item_id', 'cat_id']
)

print(f"Demand statistics calculated for {len(demand_stats)} items")
demand_stats.head()

Demand statistics calculated for 3049 items


,store_id,item_id,cat_id,sales_sum,sales_mean,sales_std,sales_min,sales_max,sales_count,revenue_sum,revenue_mean,sell_price_mean,sell_price_min,sell_price_max,demand_cv,fill_rate
0,CA_1,FOODS_1_001,FOODS,290,0.641593,1.016426,0,7,452,649.60,1.437168,2.24,2.24,2.24,1.584223,0.398230
1,CA_1,FOODS_1_002,FOODS,193,0.426991,0.739435,0,4,452,1829.64,4.047876,9.48,9.48,9.48,1.731734,0.311947
2,CA_1,FOODS_1_003,FOODS,326,0.721239,1.146552,0,9,452,1052.98,2.329602,3.23,3.23,3.23,1.589698,0.420354
3,CA_1,FOODS_1_004,FOODS,4589,10.152655,8.920003,0,48,452,8994.44,19.899204,1.96,1.96,1.96,0.878588,0.780973
4,CA_1,FOODS_1_005,FOODS,486,1.075221,1.738722,0,24,452,1720.44,3.806283,3.54,3.54,3.54,1.617083,0.528761


In [ ]:
# Demand distribution
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

# Average demand
axes[0, 0].hist(demand_stats['sales_mean'], bins=50, alpha=0.7, color='steelblue', edgecolor='black')
axes[0, 0].set_title('Average Daily Demand Distribution', fontweight='bold')
axes[0, 0].set_xlabel('Average Daily Sales')
axes[0, 0].set_ylabel('Number of Items')
axes[0, 0].axvline(demand_stats['sales_mean'].median(), color='red', linestyle='--', 
                   linewidth=2, label=f'Median: {demand_stats["sales_mean"].median():.2f}')
axes[0, 0].legend()
axes[0, 0].grid(True, alpha=0.3)

# Coefficient of variation
axes[0, 1].hist(demand_stats['demand_cv'].clip(0, 5), bins=50, alpha=0.7, color='orange', edgecolor='black')
axes[0, 1].set_title('Demand Variability (CV) Distribution', fontweight='bold')
axes[0, 1].set_xlabel('Coefficient of Variation')
axes[0, 1].set_ylabel('Number of Items')
axes[0, 1].axvline(0.5, color='green', linestyle='--', linewidth=2, label='Low variability threshold')
axes[0, 1].axvline(1.0, color='red', linestyle='--', linewidth=2, label='High variability threshold')
axes[0, 1].legend()
axes[0, 1].grid(True, alpha=0.3)

# Revenue distribution
axes[1, 0].hist(demand_stats['revenue_sum'] / 1000, bins=50, alpha=0.7, color='green', edgecolor='black')
axes[1, 0].set_title('Total Revenue Distribution', fontweight='bold')
axes[1, 0].set_xlabel('Revenue ($1000s)')
axes[1, 0].set_ylabel('Number of Items')
axes[1, 0].grid(True, alpha=0.3)

# Fill rate
axes[1, 1].hist(demand_stats['fill_rate'], bins=50, alpha=0.7, color='purple', edgecolor='black')
axes[1, 1].set_title('Fill Rate Distribution', fontweight='bold')
axes[1, 1].set_xlabel('Fill Rate (% of days with sales)')
axes[1, 1].set_ylabel('Number of Items')
axes[1, 1].axvline(demand_stats['fill_rate'].median(), color='red', linestyle='--', 
                   linewidth=2, label=f'Median: {demand_stats["fill_rate"].median():.2%}')
axes[1, 1].legend()
axes[1, 1].grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Scatter: Demand vs Variability by category
fig, ax = plt.subplots(figsize=(14, 8))

for cat in demand_stats['cat_id'].unique():
    cat_data = demand_stats[demand_stats['cat_id'] == cat]
    ax.scatter(cat_data['sales_mean'], cat_data['demand_cv'].clip(0, 5), 
              alpha=0.5, s=50, label=cat)

ax.set_xlabel('Average Daily Demand', fontsize=12)
ax.set_ylabel('Coefficient of Variation (CV)', fontsize=12)
ax.set_title('Demand vs Variability by Category', fontsize=14, fontweight='bold')
ax.axhline(y=0.5, color='green', linestyle='--', alpha=0.5, label='Low variability')
ax.axhline(y=1.0, color='red', linestyle='--', alpha=0.5, label='High variability')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## 7. Key Insights Summary

In [ ]:
print("=" * 70)
print("KEY INSIGHTS FROM DATA EXPLORATION")
print("=" * 70)
print()
print(f"📊 Data Coverage:")
print(f"  - Total products: {len(sales_train):,}")
print(f"  - Stores analyzed: CA_1")
print(f"  - Time period: 2015-01-01 to 2016-03-27")
print(f"  - Total records: {len(store_data):,}")
print()
print(f"📈 Demand Characteristics:")
print(f"  - Average daily demand: {demand_stats['sales_mean'].mean():.2f} units")
print(f"  - Median CV: {demand_stats['demand_cv'].median():.2f}")
print(f"  - Items with low variability (CV < 0.5): {(demand_stats['demand_cv'] < 0.5).sum()}")
print(f"  - Items with high variability (CV > 1.0): {(demand_stats['demand_cv'] > 1.0).sum()}")
print()
print(f"💰 Revenue Insights:")
print(f"  - Total revenue: ${demand_stats['revenue_sum'].sum():,.2f}")
print(f"  - Average item revenue: ${demand_stats['revenue_sum'].mean():,.2f}")
print(f"  - Top revenue item: ${demand_stats['revenue_sum'].max():,.2f}")
print()
print(f"💵 Price Characteristics:")
print(f"  - Average price: ${store_data['sell_price'].mean():.2f}")
print(f"  - Price range: ${store_data['sell_price'].min():.2f} - ${store_data['sell_price'].max():.2f}")
print()
print("✅ Ready for ABC/XYZ classification and inventory optimization!")
print("=" * 70)

## Next Steps

1. **ABC/XYZ Classification**: Move to notebook `02_abc_xyz_analysis.ipynb`
2. **Inventory Optimization**: See notebook `03_inventory_optimization.ipynb`
3. **Save processed data** for use in subsequent analyses

In [20]:
# Save processed data for next notebooks
output_path = project_root / 'data/processed'
output_path.mkdir(parents=True, exist_ok=True)

demand_stats.to_csv(output_path / 'demand_statistics_ca1.csv', index=False)
print(f"✅ Saved demand statistics to {output_path / 'demand_statistics_ca1.csv'}")

✅ Saved demand statistics to /Users/godsonkurishinkal/Projects/data-science-portfolio/project-002-inventory-optimization-engine/data/processed/demand_statistics_ca1.csv
